In [24]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt  # Visualization
import warnings  # Supress warnings
import os
import random

warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
from keras.models import load_model

df = pd.read_csv("./files/scaled_data.csv", index_col=0)
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
df

,Rainfall,Depth_to_Groundwater,Temperature,Drainage_Volume,River_Hydrometry
Date,,,,,
2009-01-01,0.0,0.175907,0.242507,0.672018,0.260870
2009-01-02,0.0,0.177960,0.163488,0.507291,0.304348
2009-01-03,0.0,0.180698,0.220708,0.624189,0.260870
2009-01-04,0.0,0.182067,0.122616,0.540839,0.260870
2009-01-05,0.0,0.184805,0.049046,0.465917,0.217391
...,...,...,...,...,...
2020-06-26,0.0,0.591376,0.801090,0.462974,0.304348
2020-06-27,0.0,0.588638,0.814714,0.408690,0.260870
2020-06-28,0.0,0.583162,0.833787,0.378186,0.260870


In [25]:
uni = df['Depth_to_Groundwater']
uni

Date
2009-01-01    0.175907
2009-01-02    0.177960
2009-01-03    0.180698
2009-01-04    0.182067
2009-01-05    0.184805
                ...   
2020-06-26    0.591376
2020-06-27    0.588638
2020-06-28    0.583162
2020-06-29    0.583847
2020-06-30    0.579055
Name: Depth_to_Groundwater, Length: 4199, dtype: float64

In [26]:
# params
WINDOW_SIZE = 5


In [27]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [28]:
from tensorboard.plugins.hparams import api as hp

In [29]:
def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size]
        y.append(label)
    return np.array(X), np.array(y)

In [30]:
X1, y1 = df_to_X_y(uni, WINDOW_SIZE)
X1.shape, y1.shape

((4194, 5, 1), (4194,))

In [31]:
X_train1, y_train1 = X1[:2000], y1[:2000]
X_val1, y_val1 = X1[2000:4000], y1[2000:4000]
X_test1, y_test1 = X1[4000:], y1[4000:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

((2000, 5, 1), (2000,), (2000, 5, 1), (2000,), (194, 5, 1), (194,))

In [32]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd' ,'nadam']))

METRIC_RMSE = 'RootMeanSquaredError'

logdir = 'logs/hparam_tuning'
with tf.summary.create_file_writer(logdir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_RMSE, display_name='RMSE')]
    )


num_units': 64, 'dropout': 0.1, 'optimizer': 'adam'

In [39]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam

def train_test_model(hparams):
    model = Sequential()
    model.add(InputLayer((5, 1)))
    model.add(LSTM(hparams[HP_NUM_UNITS]))
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    model.add(Dense(8, 'relu'))
    model.add(Dense(1, 'linear'))
    model.compile(loss=MeanSquaredError(), optimizer=hparams[HP_OPTIMIZER], metrics=['RootMeanSquaredError'])
    model.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=10, callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ])
    _, rmse = model.evaluate(X_test1, y_test1)
    return  rmse

In [40]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        rmse = train_test_model(hparams)
        tf.summary.scalar(METRIC_RMSE, rmse, step=10)


In [41]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1


--- Starting trial: run-0
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/10
63/63 [==============================] - 4s 16ms/step - loss: 0.0296 - root_mean_squared_error: 0.1721 - val_loss: 5.7164e-04 - val_root_mean_squared_error: 0.0239
Epoch 2/10
63/63 [==============================] - 0s 7ms/step - loss: 0.0012 - root_mean_squared_error: 0.0348 - val_loss: 5.5658e-05 - val_root_mean_squared_error: 0.0075
Epoch 3/10
63/63 [==============================] - 0s 7ms/step - loss: 2.6623e-04 - root_mean_squared_error: 0.0163 - val_loss: 1.0346e-04 - val_root_mean_squared_error: 0.0102
Epoch 4/10
63/63 [==============================] - 1s 8ms/step - loss: 2.0754e-04 - root_mean_squared_error: 0.0144 - val_loss: 9.6079e-05 - val_root_mean_squared_error: 0.0098
Epoch 5/10
63/63 [==============================] - 1s 10ms/step - loss: 1.7848e-04 - root_mean_squared_error: 0.0134 - val_loss: 9.4563e-05 - val_root_mean_squared_error: 0.0097
Epoch 6/10
63/63 [=================

{'num_units': 32, 'dropout': 0.1, 'optimizer': 'nadam'} 0.0933
{'num_units': 64, 'dropout': 0.1, 'optimizer': 'nadam'} 0.08706
{'num_units': 64, 'dropout': 0.2, 'optimizer': 'adam'} 0.0856

In [42]:
%tensorboard --logdir logs/hparam_tuning

Launching TensorBoard...

In [43]:
import winsound
winsound.Beep(2500,1000)